<a href="https://colab.research.google.com/github/abhipsitabose/LTC-Rota-Optimisation/blob/main/Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sat Aug 16 23:44:11 2025

@author: abhip
"""

# STEP 1: Imports
import pandas as pd
import numpy as np
import random
import json
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict, Counter
from pulp import *
from deap import base, creator, tools, algorithms
import time
import os

# STEP 2: Load Data (adjust paths if needed)
try:
    data_folder = r"C:\Users\abhip\OneDrive\Desktop\Warwick\Dissertation\Data 2"

    staff_df = pd.read_csv(os.path.join(data_folder, "staff.csv"))
    absences_df = pd.read_csv(os.path.join(data_folder, "absences.csv"))
    demand_df = pd.read_csv(os.path.join(data_folder, "demand.csv"))
    costs_df = pd.read_csv(os.path.join(data_folder, "costs.csv"))

    print("All files loaded successfully.")
except Exception as e:
    print(f"Data loading error: {e}")


# STEP 3: Parse staff preferences
def parse_pref_shifts(val):
    try:
        prefs = json.loads(val)
        return {(p['day'], p['shift']) for p in prefs}
    except:
        return set()

staff_prefs = {row.staff_id: parse_pref_shifts(row.pref_shifts) for row in staff_df.itertuples(index=False)}

# STEP 4: Constants
SHIFT_HOURS = 8
roles = costs_df['role'].unique()
POP_SIZE, N_GEN, CX_PB, MUT_PB = 40, 60, 0.6, 0.4
staff_sizes = [50, 100, 150]
absence_rates = [0.05, 0.10, 0.12]
preference_levels = {'low': 0.1, 'medium': 0.5, 'high': 1.0}
agency_multipliers = [1.5, 2.0]

# STEP 5: Evaluation function
def evaluate_schedule(schedule_df, staff_df_input, demand_df_input):
    merged = schedule_df.merge(staff_df_input[['staff_id','role','max_hours_week','base_rate']], on='staff_id', how='left')
    merged['hours'] = SHIFT_HOURS
    merged['week'] = ((merged['day'] - 1)//7) + 1

    demand_long = [{'day': row['day_number'], 'shift': row['shift'], 'role': role, 'required': row.get(f"req_{role}", 0)}
                   for _, row in demand_df_input.iterrows() for role in roles]
    demand_exp = pd.DataFrame(demand_long)
    assigned = merged.groupby(['day','shift','role']).size().reset_index(name='assigned')
    coverage_check = pd.merge(demand_exp, assigned, how='left', on=['day','shift','role']).fillna(0)
    total_req = coverage_check['required'].sum()
    total_unfilled = (coverage_check['required'] - coverage_check['assigned']).clip(lower=0).sum()
    coverage_rate = 100 * (1 - total_unfilled / total_req) if total_req else 100

    weekly = merged.groupby(['staff_id','week']).agg(shifts=('shift','count'), weekly_hours=('hours','sum'), max_hours=('max_hours_week','first')).reset_index()
    weekly['utilisation_pct'] = 100 * weekly['weekly_hours'] / weekly['max_hours']
    avg_util = weekly['utilisation_pct'].mean()

    staff_totals = merged.groupby('staff_id').agg(total_hours=('hours','sum'), total_shifts=('shift','count')).reset_index()
    std_hours = staff_totals['total_hours'].std()
    std_shifts = staff_totals['total_shifts'].std()

    merged['shift_cost'] = merged['hours'] * merged['base_rate']
    total_cost = merged['shift_cost'].sum()

    overtime_hours = (weekly[weekly['weekly_hours'] > weekly['max_hours']]['weekly_hours'] - weekly[weekly['weekly_hours'] > weekly['max_hours']]['max_hours']).sum()

    pref_hit = schedule_df.apply(lambda row: (row['day'], row['shift']) in staff_prefs.get(row['staff_id'], set()), axis=1)
    pref_hit_rate = 100 * pref_hit.sum() / len(schedule_df) if len(schedule_df) else 0

    return {
        'coverage': coverage_rate,
        'avg_utilisation': avg_util,
        'std_hours': std_hours,
        'std_shifts': std_shifts,
        'total_cost': total_cost,
        'overtime_hours': overtime_hours,
        'pref_hit_rate': pref_hit_rate
    }

# --- Timing Helper ---
def timed_run(scheduler_func, *args):
    start = time.time()
    schedule, metrics = scheduler_func(*args)
    metrics['runtime_sec'] = round(time.time() - start, 2)
    return schedule, metrics

# --- Method Tagging ---
def add_method(metrics_dict, method_label):
    metrics_dict['method'] = method_label
    return metrics_dict

# ---- Placeholder for MIP and GA functions ----

# STEP 7 - MIP Scheduling
def run_mip_schedule(staff_df_input, demand_df_input, costs_df_input,
                     absences_df_input, preference_weight, agency_multiplier):

    model = LpProblem("Baseline_Scheduling", LpMinimize)
    staff_ids = staff_df_input['staff_id'].unique()
    days = demand_df_input['day_number'].unique()
    shifts = demand_df_input['shift'].unique()
    roles = costs_df_input['role'].unique()

    cost_map = staff_df_input.set_index('staff_id')['base_rate'].to_dict()
    x = LpVariable.dicts("assign", (staff_ids, days, shifts), cat=LpBinary)

    unmet = {}
    for _, row in demand_df_input.iterrows():
        d, s = row['day_number'], row['shift']
        for role in roles:
            unmet[(d, s, role)] = LpVariable(f"unmet_d{d}_s{s}_r{role}", lowBound=0, cat='Integer')

    PENALTY_UNMET = 1 * agency_multiplier * SHIFT_HOURS * max(costs_df_input['base_rate'])
    model += (
        lpSum(x[i][d][s] * cost_map.get(i, 0) * SHIFT_HOURS for i in staff_ids for d in days for s in shifts) +
        lpSum(PENALTY_UNMET * unmet[(d, s, role)] for (d, s, role) in unmet)
    )

    for i in staff_ids:
        for d in days:
            model += lpSum(x[i][d][s] for s in shifts) <= 1

    for _, row in demand_df_input.iterrows():
        d, s = row['day_number'], row['shift']
        for role in roles:
            req = row[f"req_{role}"]
            eligible = staff_df_input[staff_df_input['role'] == role]['staff_id']
            model += lpSum(x[i][d][s] for i in eligible) + unmet[(d, s, role)] >= req

    for i in staff_ids:
        max_hours = staff_df_input.loc[staff_df_input['staff_id'] == i, 'max_hours_week'].values[0]
        for w in range(1, 5):
            days_in_week = [d for d in days if ((d - 1) // 7 + 1) == w]
            model += lpSum(x[i][d][s] * SHIFT_HOURS for d in days_in_week for s in shifts) <= max_hours

    rest_hours_map = staff_df_input.set_index('staff_id')['min_rest_hours'].to_dict()
    for i in staff_ids:
        for d in days:
            if d < max(days):
                model += x[i][d]['N'] + x[i][d+1]['D'] <= 1
                if rest_hours_map.get(i, 8) >= 12:
                    model += x[i][d]['E'] + x[i][d+1]['D'] <= 1

    weekend_days = demand_df_input[demand_df_input['is_weekend'] == True]['day_number'].unique()
    for i in staff_ids:
        model += lpSum(x[i][d][s] for d in weekend_days for s in shifts) >= 2

    model.solve(PULP_CBC_CMD(msg=0))

    schedule_list = [(i, d, s) for i in staff_ids for d in days for s in shifts if x[i][d][s].varValue and x[i][d][s].varValue > 0.5]
    schedule_df = pd.DataFrame(schedule_list, columns=['staff_id', 'day', 'shift'])
    metrics = evaluate_schedule(schedule_df, staff_df_input, demand_df_input)
    return schedule_df, metrics


# STEP 8 - GA Refinement
from deap import base, creator, tools, algorithms

def run_ga_refinement(mip_schedule_df, staff_df_input, demand_df_input,
                      absences_df_input, preference_weight):

    # Prepare slot list
    roles = ['RN', 'LPN', 'PSW']
    slot_rows = []
    for _, row in demand_df_input.iterrows():
        d, s = int(row['day_number']), row['shift']
        for r in roles:
            for _ in range(int(row[f"req_{r}"])):
                slot_rows.append((d, s, r))
    slots_df = pd.DataFrame(slot_rows, columns=['day', 'shift', 'role'])
    num_slots = len(slots_df)

    # Staff lookup maps
    availability = {(int(r.staff_id), int(r.day_number)): (int(r.absent) == 0)
                    for r in absences_df_input.itertuples(index=False)}
    role_by_staff = staff_df_input.set_index('staff_id')['role'].to_dict()
    max_week_hours = staff_df_input.set_index('staff_id')['max_hours_week'].to_dict()
    base_rate = staff_df_input.set_index('staff_id')['base_rate'].to_dict()
    eligible_by_role = {r: staff_df_input.loc[staff_df_input['role'] == r, 'staff_id'].tolist()
                        for r in roles}

    weeks_by_day = {d: ((d - 1) // 7) + 1 for d in demand_df_input['day_number'].unique()}

    # Feasibility + repair functions
    def feasible_ind(genes):
        per_day = defaultdict(set)
        per_week_hours = defaultdict(float)
        for idx, staff in enumerate(genes):
            d, s, r = slots_df.iloc[idx]
            if role_by_staff.get(staff) != r: return False
            if not availability.get((staff, int(d)), True): return False
            if int(d) in per_day[staff]: return False
            per_day[staff].add(int(d))
            w = weeks_by_day[d]
            per_week_hours[(staff, w)] += SHIFT_HOURS
            if per_week_hours[(staff, w)] > max_week_hours.get(staff, 1e9):
                return False
        return True

    def random_feasible_staff(d, s, r, day_used_by_staff, week_hours_by_staff):
        candidates = eligible_by_role[r]
        random.shuffle(candidates)
        w = weeks_by_day[d]
        for cand in candidates:
            if not availability.get((cand, int(d)), True): continue
            if int(d) in day_used_by_staff[cand]: continue
            if week_hours_by_staff[(cand, w)] + SHIFT_HOURS <= max_week_hours.get(cand, 1e9):
                return cand
        return None

    def repair(genes, max_attempts=3):
        day_used_by_staff = defaultdict(set)
        week_hours_by_staff = defaultdict(float)
        for idx, staff in enumerate(genes):
            d, s, r = slots_df.iloc[idx]
            day_used_by_staff[staff].add(int(d))
            week_hours_by_staff[(staff, weeks_by_day[d])] += SHIFT_HOURS
        for _ in range(max_attempts):
            changed = False
            seen_day_staff = defaultdict(set)
            for idx, staff in enumerate(genes):
                d, s, r = slots_df.iloc[idx]
                w = weeks_by_day[d]
                if role_by_staff.get(staff) != r or not availability.get((staff, int(d)), True) \
                    or int(d) in seen_day_staff[staff] \
                    or week_hours_by_staff[(staff, w)] > max_week_hours.get(staff, 1e9):
                    alt = random_feasible_staff(d, s, r, day_used_by_staff, week_hours_by_staff)
                    if alt is not None:
                        if int(d) in day_used_by_staff[staff]:
                            day_used_by_staff[staff].discard(int(d))
                            week_hours_by_staff[(staff, w)] -= SHIFT_HOURS
                        genes[idx] = alt
                        day_used_by_staff[alt].add(int(d))
                        week_hours_by_staff[(alt, w)] += SHIFT_HOURS
                        changed = True
                seen_day_staff[staff].add(int(d))
            if not changed: break
        return genes

    # Initial pop from MIP + random
    baseline_assignments = []
    merged_baseline = mip_schedule_df.merge(staff_df_input[['staff_id','role']], on='staff_id', how='left')
    cursor_counter = Counter()
    baseline_by_slot = defaultdict(list)
    for r in merged_baseline.itertuples(index=False):
        baseline_by_slot[(r.day, r.shift, r.role)].append(r.staff_id)
    for d, s, r in slots_df.itertuples(index=False):
        idx = cursor_counter[(d, s, r)]
        if idx < len(baseline_by_slot[(d, s, r)]):
            baseline_assignments.append(baseline_by_slot[(d, s, r)][idx])
        else:
            baseline_assignments.append(random.choice(eligible_by_role[r]))
        cursor_counter[(d, s, r)] += 1
    baseline_assignments = repair(baseline_assignments, max_attempts=5)

    # Fitness
    creator.create("FitnessMulti", base.Fitness,
                   weights=(-1.0, -0.5, preference_weight))
    creator.create("Individual", list, fitness=creator.FitnessMulti)

    def compute_totals(genes):
        total_hours = Counter()
        pref_hits = 0
        for idx, staff in enumerate(genes):
            d, s, r = slots_df.iloc[idx]
            total_hours[staff] += SHIFT_HOURS
            if (int(d), s) in staff_prefs.get(staff, set()):
                pref_hits += 1
        total_cost = sum(hr * base_rate.get(staff, 0) for staff, hr in total_hours.items())
        pref_score = pref_hits / len(genes)
        return total_hours, total_cost, pref_score

    def fitness_fn(ind):
        if not feasible_ind(ind):
            return (1e12, 1e6, 0.0)
        totals, cost, pref_score = compute_totals(ind)
        std_hours = np.std(list(totals.values()) or [0.0])
        return (cost, std_hours, pref_score)

    toolbox = base.Toolbox()
    toolbox.register("individual", creator.Individual, baseline_assignments.copy())
    def random_ind():
        ind = [random.choice(eligible_by_role[r]) for _, s, r in slots_df.itertuples(index=False)]
        return creator.Individual(repair(ind, 5))
    toolbox.register("population", tools.initCycle, list,
                     (toolbox.individual, random_ind), n=POP_SIZE//2)
    toolbox.register("evaluate", fitness_fn)

    def cx_uniform(ind1, ind2, indpb=0.2):
        for i in range(len(ind1)):
            if random.random() < indpb:
                ind1[i], ind2[i] = ind2[i], ind1[i]
        ind1[:] = repair(ind1, 2)
        ind2[:] = repair(ind2, 2)
        return ind1, ind2

    def mut_reassign(ind, prob=0.08):
        for i in range(len(ind)):
            if random.random() < prob:
                d, s, r = slots_df.iloc[i]
                alt = random_feasible_staff(d, s, r, defaultdict(set), defaultdict(float))
                if alt is not None:
                    ind[i] = alt
        ind[:] = repair(ind, 2)
        return (ind,)

    toolbox.register("mate", cx_uniform, indpb=0.15)
    toolbox.register("mutate", mut_reassign, prob=0.06)
    toolbox.register("select", tools.selTournament, tournsize=3)

    pop = toolbox.population(n=POP_SIZE)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean, axis=0)
    stats.register("min", np.min, axis=0)
    stats.register("max", np.max, axis=0)

    algorithms.eaSimple(pop, toolbox, cxpb=CX_PB, mutpb=MUT_PB, ngen=N_GEN,
                        stats=stats, halloffame=hof, verbose=False)

    best = hof[0]
    ga_assignments = [(int(staff), int(d), s) for (staff, (d, s, _)) in zip(best, slots_df.itertuples(index=False))]
    ga_schedule_df = pd.DataFrame(ga_assignments, columns=['staff_id', 'day', 'shift'])
    metrics_ga = evaluate_schedule(ga_schedule_df, staff_df_input, demand_df_input)
    return ga_schedule_df, metrics_ga

# STEP 6: Run Experiments
results = []
backup_file = "results_summary.csv"

for staff_size in staff_sizes:
    staff_sub = staff_df.iloc[:staff_size].copy()
    absences_sub = absences_df[absences_df['staff_id'].isin(staff_sub['staff_id'])].copy()

    for absence_rate in absence_rates:
        abs_sim = absences_sub.copy()
        num_rec = len(abs_sim)
        absent_idx = np.random.choice(abs_sim.index, size=int(num_rec * absence_rate), replace=False)
        abs_sim['absent'] = 0
        abs_sim.loc[absent_idx, 'absent'] = 1

        for pref_level, pref_weight in preference_levels.items():
            for agency_multiplier in agency_multipliers:
                print(f"▶ Running: Size={staff_size}, Abs={absence_rate}, Pref={pref_level}, Agency={agency_multiplier}")
                try:
                    mip_sched, mip_metrics = timed_run(run_mip_schedule, staff_sub, demand_df, costs_df, abs_sim, pref_weight, agency_multiplier)
                    mip_metrics = add_method(mip_metrics, "MIP")

                    ga_sched, ga_metrics = timed_run(run_ga_refinement, mip_sched, staff_sub, demand_df, abs_sim, pref_weight)
                    ga_metrics = add_method(ga_metrics, "GA")

                    agency_cost_saving = mip_metrics['total_cost'] - ga_metrics['total_cost']
                    agency_cost_saving_pct = 100 * agency_cost_saving / mip_metrics['total_cost'] if mip_metrics['total_cost'] else 0
                    pref_improvement_pct = ga_metrics['pref_hit_rate'] - mip_metrics['pref_hit_rate']

                    results.append({
                        'staff_size': staff_size,
                        'absence_rate': absence_rate,
                        'preference_level': pref_level,
                        'agency_multiplier': agency_multiplier,
                        'mip_coverage': mip_metrics['coverage'],
                        'mip_cost': mip_metrics['total_cost'],
                        'mip_pref_hit_rate': mip_metrics['pref_hit_rate'],
                        'mip_fairness_std': mip_metrics['std_hours'],
                        'mip_overtime_hours': mip_metrics['overtime_hours'],
                        'mip_runtime_sec': mip_metrics['runtime_sec'],
                        'ga_coverage': ga_metrics['coverage'],
                        'ga_cost': ga_metrics['total_cost'],
                        'ga_pref_hit_rate': ga_metrics['pref_hit_rate'],
                        'ga_fairness_std': ga_metrics['std_hours'],
                        'ga_overtime_hours': ga_metrics['overtime_hours'],
                        'ga_runtime_sec': ga_metrics['runtime_sec'],
                        'agency_cost_saving_pct': agency_cost_saving_pct,
                        'preference_improvement_pct': pref_improvement_pct
                    })

                    pd.DataFrame(results).to_csv(backup_file, index=False)

                except Exception as e:
                    print(f"ERROR: {e}")

print("Experiment complete. Results saved.")

# STEP 7: Plot Results
try:
    df_results = pd.read_csv("results_summary.csv")

    plt.figure(figsize=(10, 6))
    sns.boxplot(data=df_results, x='staff_size', y='agency_cost_saving_pct')
    plt.title("% Agency Cost Saving: GA vs MIP")
    plt.ylabel("Cost Saving (%)")
    plt.xlabel("Staff Size")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(10, 6))
    sns.lineplot(data=df_results, x='staff_size', y='preference_improvement_pct', hue='preference_level', marker='o')
    plt.title("% Preference Improvement by GA")
    plt.ylabel("Improvement (%)")
    plt.xlabel("Staff Size")
    plt.grid(True)
    plt.tight_layout()
    plt.show()
except Exception as e:
    print(f"Plotting error: {e}")
